In [4]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset, Dataset
import torch
import re ,os
import pandas as pd 



c:\Users\LeeGyeSeung\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 1. KoBART 모델과 토크나이저 로드
model_name = 'gogamza/kobart-summarization'
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)

In [7]:
# 데이터 로드 (CSV 파일, 컬럼: "input", "summary")

merged_df = pd.read_csv('combined_summary_data_utf-8.csv')

import re

merged_df

,input,summary
0,제1조 [목적] 이 계약서의 작성 목적은 갑과 을 간 상품의 직매입거래(갑이 매입 ...,제1조 [목적] 이 조는 ‘갑’과 ‘을’이 상품을 직매입거래(‘갑’이 상품을 미리 ...
1,제2조 [기본원칙] ① 갑과 을은 상호이익을 존중하고 이 계약상 의무를 신의에 따라...,제2조 [기본원칙] 갑과 을은 상호 이익을 존중하고 신의성실 원칙에 따라 공정한 거...
2,제3조 [공정거래 준수 및 동반성장 지원] ① 갑과 을은 공정하고 투명한 거래를 위...,제3조 [공정거래 준수 및 동반성장 지원] 갑과 을은 금품·향응 제공이나 우월적 지...
3,"제4조 [상품의 납품] ① 을이 갑에게 납품하여야 하는 상품의 종류, 수량 및 가격...",제4조 [상품의 납품] 을은 별지 또는 관련 문서를 통해 정해진 상품 종류·수량·가...
4,제5조 [검수기준 및 품질검사] ① 갑은 납품받는 상품에 관하여 갑의 검수기준에 따...,제5조 [검수기준 및 품질검사] 갑은 사전에 고지한 검수기준에 따라 납품상품을 검사...
...,...,...
211,"제25조 [상계] 이 계약이 해지될 경우, 이 계약과 관련하여 해지 시점까지 발생하...","제25조 [상계] 계약이 해지되면, 해지 시점까지 발생한 이미 이행 기한이 지난 채..."
212,제26조 [계약의 유효기간 및 갱신] ① 이 계약의 유효기간은 2025. 01. 0...,제26조 [계약의 유효기간 및 갱신] 계약 유효기간은 2025. 01. 01.부터 ...
213,제26조의2 [계약의 갱신 기준 등의 사전 통지] ① 갑은 계약 갱신 거절 기준을 ...,"제26조의2 [계약의 갱신 기준 등의 사전 통지] ""갑""은 계약 갱신 거절 기준을 ..."
214,제27조 [분쟁해결 및 재판관할] ① 이 계약에서 정하지 않은 사항 또는 계약 내용...,"제27조 [분쟁해결 및 재판관할] ""갑""과 ""을""은 합의에 따르며, 합의가 안 될 ..."


In [8]:

# 공백 2개 이상을 1개로 변환하는 함수
def clean_whitespace(text):
    text = text.replace('“','').replace('”','').replace("'",'').replace('"','')
    return re.sub(r'\s{2,}', ' ', text.strip())


merged_df['input'] = merged_df['input'].apply(clean_whitespace)
merged_df['summary'] = merged_df['summary'].apply(clean_whitespace)
merged_df['article'] = merged_df['input'].apply(lambda x: x.split('[')[0].strip())

merged_df['input'] = "다음 조항을 핵심 내용 위주로 요약하세요: " + merged_df['input'] 
#merged_df['input'] = "다음 조항을 전체적인 내용을 잘 반영하도록 1~3줄로 요약하세요 " + merged_df['input'] 
print(merged_df['input'][8])

다음 조항을 핵심 내용 위주로 요약하세요: 제8조 [판매장려금] ① 갑은 상품의 판매촉진을 위하여 다음 조건에 따라 을로부터 판매장 려금을 수령할 수 있다. 명칭 종류 지급목적 지급시기 지급횟수 판매장려금률(액) 판매 인센티브 성과장려금 갑이 일정한 양 이상의 상품을 성과 달성 시 연 1회 초과판매액의 5% ② 제1항의 판매장려금률(액)의 결정기준ㆍ결정절차 및 변경사유ㆍ변경기준ㆍ변 경절차는 각각 다음과 같다. 1. 결정기준: 소비자 불만 및 판매 성과 부진 등의 이유로 결정 2. 결정절차: 1)반품 요청은 갑에서 상품을 수령한 후 7일 이내에 서면으로 제출 2) 을은 반품 요청서를 접수한 후 3영업일 이내에 검토 후 승인 여부를 통보 3. 변경사유: 반품 조건이나 반품 가능 기간의 변경은 소비자의 구매 성향 변 화, 제품의 품질 불만, 제품 유통기한 경과 등 4. 변경기준: 반품 기한과 반품 수량은 갑과 을 간의 합의에 따라, 신상품 출시 또는 특별 프로모션에 의해 변경 5. 변경절차: 갑은 서면 통지로 변경 내용을 을에게 알리며, 양 당사자는 반 품에 대한 세부 사항을 재협상하여 합의된 후 반영, 변경된 조건은 통지 후 7일 이내에 적용 판매 시 판매독려 우수 시즌 판매 인센 티브 시즌장려금 갑이 을의 한정판 제품에 한 하여 실적 향상 시즌 종 료 시 분기별 1 회 총 판매액의 1% ※ 결정기준및변경기준- 갑과을 간의상품거래량및각당사자의상대방에 대한매출액이전체매출액에서차지하는비중, 거래기간, 갑의판매장려금정책 등을종합적으로감안하여실제부과하는판매장려금의수준을결정·변경하기 위한기준을기재 ※ 결정절차및변경절차- 담당MD와납품업체간의협의절차, 협의불성립시 재협의절차, 결정된판매장려금수준의통지및확인등과같은절차적사항을 기재 ※ 변경사유- 매출액의증감, 시장상황의변동과같이판매장려금률의인상/인하가 필요한사유를기재 ③ 갑은 계약기간 중 정당한 사유 없이 제1항의 판매장려금과 관련한 사항을 을에게 불리하게 변경하거나, 을에게 제1항의 판매장려금 외에 자기 또는 제3자를 위하여

In [9]:

import re
import unicodedata

def clean_text(text):
    text = unicodedata.normalize('NFKC', text)  # 유니코드 정규화
    text = re.sub(r'[^\uAC00-\uD7A3a-zA-Z0-9\s]', '', text)  # 한글, 영어, 숫자, 공백 제외 특수 문자 제거
    return text

merged_df['input'] = merged_df['input'].apply(clean_text)
merged_df['summary'] = merged_df['summary'].apply(clean_text)


In [10]:
import re
#print(merged_df['input'][1])

def convert_special_numbers(text):
    special_numbers = {
        '①': '(1)', '②': '(2)', '③': '(3)', '④': '(4)', '⑤': '(5)', 
        '⑥': '(6)', '⑦': '(7)', '⑧': '(8)', '⑨': '(9)', '⑩': '(10)'
    }
    for k, v in special_numbers.items():
        text = text.replace(k, v)
    return text

# 적용 예시
cleaned_text = convert_special_numbers(merged_df['input'][1])
cleaned_text

# merged_df의 input 컬럼에 변환 함수 적용
merged_df['input'] = merged_df['input'].apply(convert_special_numbers)


In [11]:
from sklearn.model_selection import train_test_split
# dataset은 "train" 키로 로드됨; Train/Test Split (예: 90% train, 10% test)
X_train,X_val,y_train,y_val = train_test_split(merged_df['input'],merged_df['summary'],test_size=0.2, random_state=42,stratify=merged_df['article'] )
train_data = pd.DataFrame({'text': X_train, 'summary': y_train})
val_data = pd.DataFrame({'text': X_val, 'summary': y_val})
print(f'X_train: {len(X_train)}, X_val: {len(X_val)}')

X_train: 172, X_val: 44


In [12]:
from datasets import Dataset

# Pandas DataFrame을 Hugging Face Dataset으로 변환
train_dataset = Dataset.from_pandas(train_data, preserve_index=False)
val_dataset = Dataset.from_pandas(val_data, preserve_index=False)

# 데이터 구조 확인
print(train_dataset)
print(val_dataset)
print(train_dataset[0])

Dataset({
    features: ['text', 'summary'],
    num_rows: 172
})
Dataset({
    features: ['text', 'summary'],
    num_rows: 44
})
{'text': '다음 조항을 핵심 내용 위주로 요약하세요 제16조 각종 불이익 제공 금지 등 갑은 을에게 다음 각 호의 어느 하나에 해당하는 행위를 해서는 아니 된 다 1 을의 상품을 무상 또는 저가로 취득하거나 요구하는 등 부당한 이익을 취득하는 행위 2 거래관계를 자신만으로 한정하거나 다른 특정사업자와 거래하는 것을 강요 또는 방해하는 행위 3 정당한 사유 없이 상품권이나 물품을 구입하도록 요구하는 행위 4 이 계약과 관련하여 인지한 쌍방의 영업비밀이나 정보를 부당하게 이용하는 행위 5 갑이 을에게 유통벤더 등 제3자를 통한 납품을 유도하는 행위 6 갑이 을의 과실 없는 상품 멸실 금액을 을에게 부담시키는 행위 7 갑의 거래상의 지위를 부당하게 이용하여 을의 경영활동에 간섭하는 행위', 'summary': '제16조 각종 불이익 제공 금지 등 갑이 을에게 상품 무상취득저가요구 특정 거래 강요 부당한 구매 요구 영업비밀 남용 제3자 납품 유도 과실 없는 손실 부담 전가 및 경영 간섭 등 부당한 불이익을 주는 행위를 금지함을 규정한다'}


In [13]:
# 4. 토크나이즈 함수 정의
def preprocess_function(examples):
    # 입력 텍스트를 토큰화 (최대 길이 2048)
    model_inputs = tokenizer(examples["text"],max_length=1024,truncation=True,padding="max_length")
    # 요약 텍스트를 토큰화 (최대 길이 512)
    labels = tokenizer(examples["summary"],max_length=256,truncation=True,padding="max_length")
    # 모델의 타깃 레이블로 토큰화된 요약 추가
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [181]:
print(train_data.columns)
print(val_data.columns)
print(train_dataset)
print(train_dataset[0])



Index(['text', 'summary'], dtype='object')
Index(['text', 'summary'], dtype='object')
Dataset({
    features: ['text', 'summary'],
    num_rows: 172
})
{'text': '다음 조항을 핵심 내용 위주로 요약하세요 제16조 각종 불이익 제공 금지 등 갑은 을에게 다음 각 호의 어느 하나에 해당하는 행위를 해서는 아니 된 다 1 을의 상품을 무상 또는 저가로 취득하거나 요구하는 등 부당한 이익을 취득하는 행위 2 거래관계를 자신만으로 한정하거나 다른 특정사업자와 거래하는 것을 강요 또는 방해하는 행위 3 정당한 사유 없이 상품권이나 물품을 구입하도록 요구하는 행위 4 이 계약과 관련하여 인지한 쌍방의 영업비밀이나 정보를 부당하게 이용하는 행위 5 갑이 을에게 유통벤더 등 제3자를 통한 납품을 유도하는 행위 6 갑이 을의 과실 없는 상품 멸실 금액을 을에게 부담시키는 행위 7 갑의 거래상의 지위를 부당하게 이용하여 을의 경영활동에 간섭하는 행위', 'summary': '제16조 각종 불이익 제공 금지 등 갑이 을에게 상품 무상취득저가요구 특정 거래 강요 부당한 구매 요구 영업비밀 남용 제3자 납품 유도 과실 없는 손실 부담 전가 및 경영 간섭 등 부당한 불이익을 주는 행위를 금지함을 규정한다'}


In [14]:
# 5. 데이터셋 전처리 적용
import datasets

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 44/44 [00:00<00:00, 1999.65 examples/s]


In [133]:
"""
# 토큰화된 데이터 디코딩해서 확인
for i in range (100):
    sample_encoded = tokenized_train_dataset[i]['input_ids']
    sample_decoded = tokenizer.decode(sample_encoded, skip_special_tokens=True)
    print(sample_decoded) 

print(sample_decoded)  # 이상한 문자가 나오는지 확인
"""

"\n# 토큰화된 데이터 디코딩해서 확인\nfor i in range (100):\n    sample_encoded = tokenized_train_dataset[i]['input_ids']\n    sample_decoded = tokenizer.decode(sample_encoded, skip_special_tokens=True)\n    print(sample_decoded) \n\nprint(sample_decoded)  # 이상한 문자가 나오는지 확인\n"

In [199]:
import torch

# GPU 사용 가능한지 확인
print("CUDA 사용 가능 여부:", torch.cuda.is_available())

# 사용 중인 GPU 확인
if torch.cuda.is_available():
    print("사용 중인 GPU:", torch.cuda.get_device_name(0))
else:
    print("GPU를 인식하지 못했습니다.")

print("PyTorch 버전:", torch.__version__)
print("CUDA 버전:", torch.version.cuda)


CUDA 사용 가능 여부: True
사용 중인 GPU: NVIDIA GeForce RTX 4060
PyTorch 버전: 2.6.0+cu118
CUDA 버전: 11.8


In [200]:
# 6. 트레이닝 파라미터 설정
training_args = Seq2SeqTrainingArguments(
    output_dir='./kobart_contract_summary',
    evaluation_strategy='epoch',  # 평가 주기를 'epoch'으로 설정
    save_strategy='epoch',  # 저장 주기를 'epoch'으로 설정하여 일치시킴
    learning_rate=3e-5,  # 적절한 learning rate 설정
    per_device_train_batch_size=8,  # GPU 메모리에 맞게 배치 크기 조정
    per_device_eval_batch_size=8,  # 평가 배치 크기
    weight_decay=0.01,  # L2 규제
    save_total_limit=3,  # 최대 3개의 체크포인트만 저장
    num_train_epochs=20, # 적절한 epoch 수 설정 (과적합 방지)
    predict_with_generate=True,  # 예측 시 생성 방식 사용
    load_best_model_at_end=True,  # 훈련 후 최고 성능 모델을 로드
    fp16=True,  # Mixed Precision 활성화로 속도 및 메모리 최적화
    gradient_accumulation_steps=2,  # 메모리 절약을 위한 그래디언트 누적 (옵션)
)

# 7. 트레이너 초기화
# 수정된 트레이너 초기화
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,  # 수정: tokenized_train_dataset 사용
    eval_dataset=tokenized_val_dataset,     # 수정: tokenized_val_dataset 사용
    tokenizer=tokenizer,
)


c:\Users\LeeGyeSeung\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\LeeGyeSeung\AppData\Local\Temp\ipykernel_12524\4237694386.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [201]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 데이터셋을 Torch 형식으로 변환하고 GPU로 이동 설정
tokenized_train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [202]:
# 8. 파인튜닝 시작
trainer.train()

# 9. 파인튜닝된 모델 저장
trainer.save_model("C:/Model/kobart_finetuning")  # 모델 저장
tokenizer.save_pretrained("C:/Model/kobart_finetuning")  # 토크나이저 저장

Epoch,Training Loss,Validation Loss
1,No log,4.962843
2,No log,3.593413
3,No log,2.785927
4,No log,2.000456
5,No log,1.359437
6,No log,0.907074
7,No log,0.640815
8,No log,0.497124
9,No log,0.422732
10,No log,0.384379


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


('C:/Model/kobart_finetuning\\tokenizer_config.json',
 'C:/Model/kobart_finetuning\\special_tokens_map.json',
 'C:/Model/kobart_finetuning\\tokenizer.json')

In [5]:
# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# 모델 경로 설정
model_path = r'C:\Model\kobart_finetuning'

# 토크나이저 및 모델 불러오기
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path)
model = BartForConditionalGeneration.from_pretrained(model_path).to(device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Using device: cuda


In [20]:
import torch
import re
import pandas as pd

# 1. 모델을 GPU로 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 2. 불필요한 문자 제거 함수
#def remove_special_chars(text):
#    # 비ASCII 문자 제거 (128 이상의 모든 문자 제거)
#    return re.sub(r'[^\x00-\x7F가-힣.,!? ]+', '', text).strip()

def remove_special_chars(text):
    # 비ASCII 문자(128 이상) 탐지
    match = re.search(r'[^\x00-\x7F]', text)
    
    # 비ASCII 문자가 발견되면 해당 위치부터 잘라냄
    if match:
        text = text[:match.start()]
    
    return text.strip()




# 3. 요약 생성 함수 (특수 문자 제거 포함)
def generate_summary(model, tokenizer, input_text):
    inputs = tokenizer(f"{input_text}", return_tensors="pt", max_length=1024, truncation=True, padding=True)
    
    # 입력 데이터를 모델과 동일한 디바이스로 이동
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    # 요약 생성
    output = model.generate(
        inputs['input_ids'],
        max_length=300,
        num_beams=5,
        length_penalty=1.5,
        early_stopping=True,
        no_repeat_ngram_size=2
    )
    
    # 디코딩 후 특수 문자 제거
    decoded_summary = tokenizer.decode(output[0], skip_special_tokens=True)
    cleaned_summary = remove_special_chars(decoded_summary)
    
    return cleaned_summary

# 4. 결과 저장을 위한 리스트 초기화
original_texts = []
predicted_summaries = []

# 5. 테스트 데이터에 대해 예측 수행 및 결과 저장
for i in range(20):  
    # 원본 텍스트 가져오기
    original_text = tokenizer.decode(tokenized_val_dataset[i]['input_ids'], skip_special_tokens=True)
    
    # 예측 수행
    predicted_summary = generate_summary(model, tokenizer, original_text)
    
    # 리스트에 결과 저장
    original_texts.append(original_text)
    predicted_summaries.append(predicted_summary)
    
    print(f"원본 텍스트: {original_text}")
    print(f"예측된 요약: {predicted_summary}")
    print("="*50)

# 6. 데이터프레임 생성 및 CSV 저장
results_df = pd.DataFrame({
    'Original_Text': original_texts,
    'Predicted_Summary': predicted_summaries
})

# CSV 파일로 저장
results_df.to_csv('summary_results.csv', index=False, encoding='utf-8-sig')
print("결과가 'summary_results.csv' 파일로 저장되었습니다.")


원본 텍스트: 다음 조항을 핵심 내용 위주로 요약하세요 제12조 매장 위치 및 면적 등 1 을의 점포명 운영층 매장위치 매장면적 및 판촉사원의 수는 별지 2와 같다 2 갑은 매출이 감소할 시 을에게 다음 각 호의 행위를 할 수 있다 1 일정한 점포의 매장에서 퇴점하는 것을 방해하거나 을의 의사에 반하여 갑의 다른 점포의 매장에 입점하게 하는 행위 2 계약기간 중에 을의 매장의 위치면적시설을 변동하여 갑의 편의에 맞추도록 요구하는 행위 3 갑과 을이 매장 위치 또는 매장 인테리어를 변경이하 설비변경이라 한다하는 경우 다음 각 호의 내용을 준수한다 1 매장 바닥 조명 벽체 등 기초시설 공사에 관한 비용은 갑이 부담한다 단 기초시설 공사는 갑의 고유사양에 의하며 을이 자신의 고유사양에 의해 기초 공사를 시행하려는 경우 추가되는 소요비용에 대해 갑과 을은 사전 서면 약정을 체결하여 비용을 분담한다 2 MD개편 매장 리뉴얼 등 갑의 사유로 설비 변경하는 경우 이에 소요되는 비용은 갑이 부담한다 다만 을의 내부 매뉴얼에 따른 설비 변경이나 좋은 위치로의 매장 이동 등 을에게도 이익이 되는 변경이 있을 경우 해당 변경에 소요되는 비용은 갑과 을이 협의하여 사전 서면 약정에 의해 공정하게 분담할 수 있다 기초시설 공사비용은 제1호에 의한다 3 제2호에서 규정하고 있는 갑의 사유로 설비 변경을 하는 경우 그 구체적인 사유 제12조의2 제1항의 매장의 위치 면적 시설의 변경기준에 따른 평가결과 등를 을에게 사전 통보하여야 한다 4 제2호에서 규정하고 있는 갑의 사유 이외에 을의 개별적인 사유예 브랜드 컨셉 변경 을이 희망하여 매장 위치를 변경하는 경우 등로 설비 변경하는 경우 이에 소요되는 비용은 갑이 부담하며 그에 따라 을은 전체 비용의 30를 추가로 부담하는 것으로 하여 상호 협의하여 정한다 기초시설 공사비용은 제1호에 의한다 5 제4호에서 규정한 을의 사유로 인한 설비 변경을 실시하는 경우 타 입점업체의 영업활동에 방해가 발생하지 않도록 공사 일정 등을 갑은 을과 사전 협

In [1]:
# 모델 평가 및 예측 수행
predictions = trainer.predict(tokenized_val)
decoded_preds = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode([label for label in predictions.label_ids], skip_special_tokens=True)
original_inputs = val_data["text"].tolist()
results_df = pd.DataFrame({
    "Input": original_inputs,
    "Predicted_Summary": decoded_preds,
    "Ground_Truth_Summary": decoded_labels
    })

NameError: name 'trainer' is not defined